# Introduction 

The below script loads data from Calls_for_service file and merges with data from max_cfs_ucr_categories file.
Following constraints are applied:
- Records year range = 2014 - 2016
- Records Disposition = RTF 
- Records CrimeType = Violent Crime

# Index

- [1. Libraries](#1.-Libraries)
- [2. Data Loading](#2.-Data-Loading)
    - [A. Load Calls-for-Service (CFS) Data](#A.-Load-Calls-for-Service-Data)
    - [B. Filter Records with Disposition 'RTF'](#B.-Filter-Records-with-Disposition-'RTF')
    - [C. Filter Records for Year 2014 to 2016](#C.-Filter-Records-for-Year-2014-to-2016)    
- [3. Load Categories Data](#3.-Load-Categories-Data)
    - [A. Filter Records for CrimeType 'Violent Crime'](#A.-Filter-Records-for-CrimeType-'Violent-Crime')
- [4. Merge CFS with Categories data](#4.-Merge-CFS-with-Categories-data)
- [5. Verify Records](#5.-Verify-Records)
- [6. Generate Lat/Long from Location](#6.-Generate-Lat/Long-from-Location)
- [7. Save CFS Data to csv](#7.-Save-CFS-Data-to-csv)
- [8. Zip with most Violent Crimes](#8.-Zip-with-most-Violent-Crimes)


# 1. Libraries

Following libraries are required for this code to run successfully.

In [142]:
# Load required libraries 
import os
import csv
import zipfile
import string
import pandas as pd
import datetime as dt
from collections import Counter

# 2. Data Loading

In this section we load following files:
- Calls_for_Service Data
- MAX_CFS_UCR_Categories Data

And perform required data manipulation steps.

## A. Load Calls-for-Service Data

In [98]:
# Set location of file
path = os.path.join("../Datasets/Raw_Data/Calls_for_Service/")
path

'../Datasets/Raw_Data/Calls_for_Service/'

In [99]:
# Get filenames
filenames = os.listdir(path)
filenames

['Calls_for_Service_2012.zip',
 'Calls_for_Service_2013.zip',
 'Calls_for_Service_2014.zip',
 'Calls_for_Service_2015.zip',
 'Calls_for_Service_2016.zip']

In [100]:
# Load data from files in list
dfs = []
for f in filenames:
    zf = zipfile.ZipFile(os.path.join(path,f)) 
    dfs.append(pd.read_csv(zf.open(str.replace(f, 'zip', 'csv')),))

# Merge all df in list
cfs_df = pd.concat(dfs, ignore_index=True)

# Change datatype of column Type_ to String
cfs_df.Type_ = cfs_df.Type_.apply(str)

In [101]:
# Display top 5 rows
cfs_df.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
0,A0000112,62A,"BURGLAR ALARM, SILEN",2C,NaN,NaN,NaN,3683627,532625,1/1/2012 0:00,...,NaN,1/1/2012 0:33,NAT,NECESSARY ACTION TAKEN,NaN,NaN,009XX Decatur St,70116.0,8,"(29.958469303316875, -90.0613152964016)"
1,A0000412,94,DISCHARGING FIREARMS,2B,NaN,NaN,NaN,3732996,562418,1/1/2012 0:00,...,1/1/2012 0:16,1/1/2012 0:30,UNF,UNFOUNDED,NaN,NaN,147XX Chef Menteur Hwy,70129.0,7,"(30.038788769111676, -89.90425047516077)"
2,A0000212,103,DISTURBANCE (OTHER),1C,NaN,NaN,NaN,3687688,548824,1/1/2012 0:01,...,1/1/2012 0:01,1/1/2012 0:19,NAT,NECESSARY ACTION TAKEN,NaN,NaN,038XX Gentilly Blvd,70122.0,3,"(30.002886229898206, -90.04791794333323)"
3,A0000712,21,COMPLAINT OTHER,1H,NaN,NaN,NaN,3670776,521242,1/1/2012 0:01,...,NaN,1/1/2012 0:20,NAT,NECESSARY ACTION TAKEN,NaN,NaN,Carondelet St & Napoleon Ave,70115.0,2,"(29.927555772946167, -90.10228161624175)"
4,A0000512,62A,"BURGLAR ALARM, SILEN",2C,NaN,NaN,NaN,3665739,549621,1/1/2012 0:01,...,1/1/2012 0:09,1/1/2012 1:55,NAT,NECESSARY ACTION TAKEN,NaN,NaN,002XX W Harrison Ave,70124.0,3,"(30.005736477457617, -90.11723146931276)"


In [102]:
# Number of records
len(cfs_df)

2252907

In [103]:
# Column Names
cfs_df.columns

Index([u'NOPD_Item', u'Type_', u'TypeText', u'Priority', u'InitialType',
       u'InitialTypeText', u'InitialPriority', u'MapX', u'MapY', u'TimeCreate',
       u'TimeDispatch', u'TimeArrive', u'TimeClosed', u'Disposition',
       u'DispositionText', u'SelfInitiated', u'Beat', u'BLOCK_ADDRESS', u'Zip',
       u'PoliceDistrict', u'Location'],
      dtype='object')

## B. Filter Records with Disposition 'RTF' 

In [104]:
cfs_df = cfs_df[cfs_df.Disposition.str.strip() == 'RTF']

In [105]:
# Number of records
len(cfs_df)

428570

In [106]:
# Display top 5 rows
cfs_df.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
64,A0006912,29S,SUICIDE,2B,NaN,NaN,NaN,37369000,3513814,1/1/2012 0:16,...,1/1/2012 0:32,1/1/2012 0:59,RTF,REPORT TO FOLLOW,NaN,NaN,049XX Major Dr,70128.0,0,"(9.235500542976004E-7, -0.000002292984766499941)"
79,A0008112,966,DRUG VIOLATIONS,1G,NaN,NaN,NaN,3680920,540222,1/1/2012 0:22,...,1/1/2012 0:23,1/1/2012 1:13,RTF,REPORT TO FOLLOW,NaN,NaN,St Bernard Ave & N Tonti St,70119.0,1,"(29.979440793120016, -90.06959870234981)"
80,A0008212,21U,UNDER AGE DRINKING V,1F,NaN,NaN,NaN,3681403,531510,1/1/2012 0:22,...,1/1/2012 0:22,1/1/2012 1:33,RTF,REPORT TO FOLLOW,NaN,NaN,003XX Bourbon St,70112.0,8,"(29.95547113098435, -90.06837696099961)"
83,A0008312,66,EXTORTION (THREATS),1F,NaN,NaN,NaN,3691753,536508,1/1/2012 0:23,...,1/1/2012 0:25,1/1/2012 1:19,RTF,REPORT TO FOLLOW,NaN,NaN,014XX Alvar St,70117.0,5,"(29.96889564722802, -90.03551677167968)"
92,A0009412,94,DISCHARGING FIREARMS,2B,NaN,NaN,NaN,3696950,533169,1/1/2012 0:28,...,1/1/2012 0:28,1/1/2012 1:36,RTF,REPORT TO FOLLOW,NaN,NaN,054XX Burgundy St,70117.0,5,"(29.959551708547103, -90.01922533580768)"


## C. Filter Records for Year 2014 to 2016

In [107]:
# Convert column type to datetime
cfs_df.TimeCreate = pd.to_datetime(cfs_df.TimeCreate)

In [108]:
# Apply filter for year range 2014-2016
cfs_df = cfs_df[(cfs_df.TimeCreate >= dt.date(2014,1,1)) \
                & (cfs_df.TimeCreate < dt.date(2017,1,1))]

In [109]:
len(cfs_df)

250441

In [110]:
# Display top 5 rows
cfs_df.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
968679,A0004414,67P,PICKPOCKET,1E,21,COMPLAINT OTHER,1H,3682941,531692,2014-01-01 00:12:25,...,01/01/2014 12:12:25 AM,01/01/2014 01:31:41 AM,RTF,REPORT TO FOLLOW,Y,8B02,007XX St Peter,NaN,8,"(29.95592569, -90.06351118)"
968690,A0005614,21P,LOST PROPERTY,0E,21P,LOST PROPERTY,0E,3681813,532089,2014-01-01 00:15:23,...,01/01/2014 12:15:23 AM,01/01/2014 12:19:05 AM,RTF,REPORT TO FOLLOW,Y,8D05,St Louis St & Bourbon St,70112.0,8,"(29.95705191, -90.06705979)"
968703,A0007014,21,COMPLAINT OTHER,1H,21,COMPLAINT OTHER,1H,3681777,532065,2014-01-01 00:24:21,...,01/01/2014 12:24:21 AM,01/01/2014 01:25:54 AM,RTF,REPORT TO FOLLOW,Y,8D05,008XX Saint Louis St,70112.0,8,"(29.95698593, -90.06717422)"
968706,A0007314,103,DISTURBANCE (OTHER),2A,103,DISTURBANCE (OTHER),2A,3682976,539535,2014-01-01 00:26:00,...,01/01/2014 12:39:46 AM,01/01/2014 01:01:21 AM,RTF,REPORT TO FOLLOW,N,5A03,019XX N Johnson St,70116.0,5,"(29.97749169, -90.06312630)"
968712,A0007914,29,DEATH,1B,24,MEDICAL,2B,3716553,558988,2014-01-01 00:28:59,...,01/01/2014 12:47:00 AM,01/01/2014 02:24:59 AM,RTF,REPORT TO FOLLOW,N,7I02,112XX S Parkwood Ct,70128.0,7,"(30.02991285, -89.95634447)"


# 3. Load Categories Data

In [111]:
# Set location of file
fname = "../Datasets/Raw_Data/MAX_CFS_UCR_Categories.xlsx"

# Load file
crime_types = pd.read_excel(fname,sheetname='Sheet1')

# Select required columns
crime_types = crime_types.ix[:,['Code','UCR MAIN','Description']]

# Rename columns
crime_types.rename(columns={'Code':'Type_','UCR MAIN':'CrimeType','Description':'Description'},inplace=True)

# Change datatype of column Type_ to String
crime_types.Type_ = crime_types.Type_.apply(str)
crime_types.Type_ = crime_types.Type_.str.strip()

## A. Filter Records for CrimeType 'Violent Crime'

In [112]:
crime_types = crime_types[crime_types.CrimeType.str.strip() == 'VIOLENT CRIME']

In [113]:
crime_types.head()

,Type_,CrimeType,Description
71,30,VIOLENT CRIME,HOMICIDE
72,30C,VIOLENT CRIME,HOMICIDE BY CUTTING
73,30D,VIOLENT CRIME,HOMICIDE DOMESTIC
74,30S,VIOLENT CRIME,HOMICIDE BY SHOOTING
75,34,VIOLENT CRIME,AGGRAVATED BATTERY


# 4. Merge CFS with Categories data

In [114]:
merged_df = pd.merge(cfs_df,crime_types,on='Type_',how='inner')

In [115]:
merged_df.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,CrimeType,Description
0,A0009014,55,AGGRAVATED CRIMINAL DAMAGE,2A,55,AGGRAVATED CRIMINAL DAMAGE,2A,3669512,540197,2014-01-01 00:35:13,...,RTF,REPORT TO FOLLOW,N,3H01,046XX Iberville St,70119.0,3,"(29.97971459, -90.10562794)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE
1,A0073014,55,AGGRAVATED CRIMINAL DAMAGE,1B,56,SIMPLE CRIMINAL DAMAGE,1D,3671068,526707,2014-01-01 12:03:57,...,RTF,REPORT TO FOLLOW,N,2R01,032XX Milan St,70125.0,2,"(29.94257507, -90.10117130)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE
2,A0288814,55,AGGRAVATED CRIMINAL DAMAGE,2A,55,AGGRAVATED CRIMINAL DAMAGE,2A,3720367,573122,2014-01-03 03:15:10,...,RTF,REPORT TO FOLLOW,N,7K04,076XX Forum Blvd,70128.0,7,"(30.06864780, -89.94374869)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE
3,A0446614,55,AGGRAVATED CRIMINAL DAMAGE,1B,94,DISCHARGING FIREARM,2B,3707680,558372,2014-01-04 12:37:35,...,RTF,REPORT TO FOLLOW,N,7R03,090XX Lake Forest Blvd,70127.0,7,"(30.02850987, -89.98440526)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE
4,A0451114,55,AGGRAVATED CRIMINAL DAMAGE,1B,55,AGGRAVATED CRIMINAL DAMAGE,1B,3718188,572520,2014-01-04 13:12:22,...,RTF,REPORT TO FOLLOW,N,7K04,139XX Wales St,70128.0,7,"(30.06706701, -89.95065976)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE


In [116]:
len(merged_df)

9101

# 5. Verify Records

In [117]:
# Check length of records with CrimeType Null
len(merged_df[merged_df.CrimeType.isnull()])

0

In [118]:
# Check length of records with CrimeType Not Null
len(merged_df[merged_df.CrimeType.notnull()])

9101

In [119]:
# Remove records with CrimeType Null
cfs_final = merged_df[merged_df.CrimeType.notnull()]

In [120]:
cfs_final.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,CrimeType,Description
0,A0009014,55,AGGRAVATED CRIMINAL DAMAGE,2A,55,AGGRAVATED CRIMINAL DAMAGE,2A,3669512,540197,2014-01-01 00:35:13,...,RTF,REPORT TO FOLLOW,N,3H01,046XX Iberville St,70119.0,3,"(29.97971459, -90.10562794)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE
1,A0073014,55,AGGRAVATED CRIMINAL DAMAGE,1B,56,SIMPLE CRIMINAL DAMAGE,1D,3671068,526707,2014-01-01 12:03:57,...,RTF,REPORT TO FOLLOW,N,2R01,032XX Milan St,70125.0,2,"(29.94257507, -90.10117130)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE
2,A0288814,55,AGGRAVATED CRIMINAL DAMAGE,2A,55,AGGRAVATED CRIMINAL DAMAGE,2A,3720367,573122,2014-01-03 03:15:10,...,RTF,REPORT TO FOLLOW,N,7K04,076XX Forum Blvd,70128.0,7,"(30.06864780, -89.94374869)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE
3,A0446614,55,AGGRAVATED CRIMINAL DAMAGE,1B,94,DISCHARGING FIREARM,2B,3707680,558372,2014-01-04 12:37:35,...,RTF,REPORT TO FOLLOW,N,7R03,090XX Lake Forest Blvd,70127.0,7,"(30.02850987, -89.98440526)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE
4,A0451114,55,AGGRAVATED CRIMINAL DAMAGE,1B,55,AGGRAVATED CRIMINAL DAMAGE,1B,3718188,572520,2014-01-04 13:12:22,...,RTF,REPORT TO FOLLOW,N,7K04,139XX Wales St,70128.0,7,"(30.06706701, -89.95065976)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE


In [121]:
cfs_final.tail()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,CrimeType,Description
9096,A1977716,43M,SIMPLE RAPE MALE VICTIM,2B,21,COMPLAINT OTHER,1H,37369000,3513814,2016-01-17 06:19:35,...,RTF,REPORT TO FOLLOW,Y,3M04,046XX Blk Paris Ave,70122.0,3,"(0, 0)",VIOLENT CRIME,SIMPLE RAPE MALE VICTIM
9097,C3235016,43M,SIMPLE RAPE MALE VICTIM,2B,21,COMPLAINT OTHER,1H,37369000,3513814,2016-03-27 18:07:11,...,RTF,REPORT TO FOLLOW,N,3T02,047XX Painters St,70122.0,3,"(0, 0)",VIOLENT CRIME,SIMPLE RAPE MALE VICTIM
9098,F1435916,43M,SIMPLE RAPE MALE VICTIM,2B,21,COMPLAINT OTHER,1H,37369000,3513814,2016-06-13 14:55:27,...,RTF,REPORT TO FOLLOW,Y,4E01,050XX Woodland Dr,70131.0,4,"(0, 0)",VIOLENT CRIME,SIMPLE RAPE MALE VICTIM
9099,G2907016,43M,SIMPLE RAPE MALE VICTIM,2B,21P,LOST PROPERTY,0E,37369000,3513814,2016-07-27 11:28:13,...,RTF,REPORT TO FOLLOW,N,8C02,003XX Royal St,70130.0,8,"(0, 0)",VIOLENT CRIME,SIMPLE RAPE MALE VICTIM
9100,J0044416,43M,SIMPLE RAPE MALE VICTIM,1A,43M,SIMPLE RAPE MALE VICTIM,1A,37369000,3513814,2016-10-01 11:55:01,...,RTF,REPORT TO FOLLOW,N,1E03,015XX Basin St,70116.0,1,"(0, 0)",VIOLENT CRIME,SIMPLE RAPE MALE VICTIM


In [122]:
cfs_final.Type_.unique()

array(['55', '64G', '34', '34S', '30S', '37', '34D', '34C', '65', '42',
       '37D', '43', '64J', '64K', '64', '65J', '30C', '42B', '30', '42M',
       '30D', '43M'], dtype=object)

In [123]:
len(cfs_final)

9101

In [124]:
min(cfs_final.TimeCreate)

Timestamp('2014-01-01 00:35:13')

In [125]:
max(cfs_final.TimeCreate)

Timestamp('2016-12-31 19:46:46')

# 6. Generate Lat/Long from Location

In [126]:
len(cfs_final[cfs_final.Location=='(0, 0)'])

1084

Remove records with (lat,lon) = (0,0)

In [127]:
# cfs_final = cfs_final[cfs_final.Location!='(0, 0)']

In [128]:
# len(cfs_final)

Remove records with (lat,lon) = null

In [129]:
cfs_final = cfs_final[cfs_final.Location.notnull()]

In [130]:
len(cfs_final)

9074

In [131]:
cfs_final['Latitude'] = [str(val).split(',')[0][1:].strip() for val in cfs_final.Location] 

In [132]:
cfs_final['Longitude'] = [str(val).split(',')[1][:-1].strip() for val in cfs_final.Location] 

In [135]:
cfs_final.reset_index(inplace=True, drop=True)

In [136]:
cfs_final.columns

Index([u'NOPD_Item', u'Type_', u'TypeText', u'Priority', u'InitialType',
       u'InitialTypeText', u'InitialPriority', u'MapX', u'MapY', u'TimeCreate',
       u'TimeDispatch', u'TimeArrive', u'TimeClosed', u'Disposition',
       u'DispositionText', u'SelfInitiated', u'Beat', u'BLOCK_ADDRESS', u'Zip',
       u'PoliceDistrict', u'Location', u'CrimeType', u'Description',
       u'Latitude', u'Longitude'],
      dtype='object')

In [139]:
cfs_final.head()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,CrimeType,Description,Latitude,Longitude
0,A0009014,55,AGGRAVATED CRIMINAL DAMAGE,2A,55,AGGRAVATED CRIMINAL DAMAGE,2A,3669512,540197,2014-01-01 00:35:13,...,N,3H01,046XX Iberville St,70119.0,3,"(29.97971459, -90.10562794)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE,29.97971459,-90.10562794
1,A0073014,55,AGGRAVATED CRIMINAL DAMAGE,1B,56,SIMPLE CRIMINAL DAMAGE,1D,3671068,526707,2014-01-01 12:03:57,...,N,2R01,032XX Milan St,70125.0,2,"(29.94257507, -90.10117130)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE,29.94257507,-90.10117130
2,A0288814,55,AGGRAVATED CRIMINAL DAMAGE,2A,55,AGGRAVATED CRIMINAL DAMAGE,2A,3720367,573122,2014-01-03 03:15:10,...,N,7K04,076XX Forum Blvd,70128.0,7,"(30.06864780, -89.94374869)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE,30.06864780,-89.94374869
3,A0446614,55,AGGRAVATED CRIMINAL DAMAGE,1B,94,DISCHARGING FIREARM,2B,3707680,558372,2014-01-04 12:37:35,...,N,7R03,090XX Lake Forest Blvd,70127.0,7,"(30.02850987, -89.98440526)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE,30.02850987,-89.98440526
4,A0451114,55,AGGRAVATED CRIMINAL DAMAGE,1B,55,AGGRAVATED CRIMINAL DAMAGE,1B,3718188,572520,2014-01-04 13:12:22,...,N,7K04,139XX Wales St,70128.0,7,"(30.06706701, -89.95065976)",VIOLENT CRIME,AGGRAVATED CRIMINAL DAMAGE,30.06706701,-89.95065976


In [137]:
cfs_final.tail()

,NOPD_Item,Type_,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,CrimeType,Description,Latitude,Longitude
9069,A1977716,43M,SIMPLE RAPE MALE VICTIM,2B,21,COMPLAINT OTHER,1H,37369000,3513814,2016-01-17 06:19:35,...,Y,3M04,046XX Blk Paris Ave,70122.0,3,"(0, 0)",VIOLENT CRIME,SIMPLE RAPE MALE VICTIM,0,0
9070,C3235016,43M,SIMPLE RAPE MALE VICTIM,2B,21,COMPLAINT OTHER,1H,37369000,3513814,2016-03-27 18:07:11,...,N,3T02,047XX Painters St,70122.0,3,"(0, 0)",VIOLENT CRIME,SIMPLE RAPE MALE VICTIM,0,0
9071,F1435916,43M,SIMPLE RAPE MALE VICTIM,2B,21,COMPLAINT OTHER,1H,37369000,3513814,2016-06-13 14:55:27,...,Y,4E01,050XX Woodland Dr,70131.0,4,"(0, 0)",VIOLENT CRIME,SIMPLE RAPE MALE VICTIM,0,0
9072,G2907016,43M,SIMPLE RAPE MALE VICTIM,2B,21P,LOST PROPERTY,0E,37369000,3513814,2016-07-27 11:28:13,...,N,8C02,003XX Royal St,70130.0,8,"(0, 0)",VIOLENT CRIME,SIMPLE RAPE MALE VICTIM,0,0
9073,J0044416,43M,SIMPLE RAPE MALE VICTIM,1A,43M,SIMPLE RAPE MALE VICTIM,1A,37369000,3513814,2016-10-01 11:55:01,...,N,1E03,015XX Basin St,70116.0,1,"(0, 0)",VIOLENT CRIME,SIMPLE RAPE MALE VICTIM,0,0


# 7. Save CFS Data to csv

In [140]:
# Save the data
fullpath = "..\\Datasets\\Final_Data\\Calls_for_Service_Cleaned.csv"
cfs_final.to_csv(fullpath, sep=',',  index = False)

# 8. Zip with most Violent Crimes

In [146]:
zip_crime_count = Counter(cfs_final.Zip)

In [147]:
zip_crime_count.most_common(5)

[(70117.0, 1044),
 (70119.0, 1001),
 (70126.0, 860),
 (70116.0, 830),
 (70127.0, 619)]